In [1]:
from resources.iot.device import IoT_mqtt_publisher, IoT_sensor
from resources.iot.device import IoT_sensor_consumer
from IPython.core.display import display
import ipywidgets as widgets
from resources.iot.device import IoT_mqtt_publisher, IoT_sensor
from IPython.core.display import HTML 
import os

# Componente para simulação de um sensor

<!--
```bash
IoT_sensor(<name/id>, <grandeza física >, <unidade de medida>, <menor valor>, <maior valor possível>, <intervalo entre leituras (segundos)>)
```

Exemplo de sensor de pressão:

```python
sensor_pressao = IoT_sensor("32", "pressao", "bar", 20, 35, 5)

```
-->

Componentes `IoT_sensor` podem se conectar a componentes do tipo `IoT_mqtt_publisher` para publicar, em um tópico, mensagens referentes às leituras feitas pelo sensor. Por exemplo, o sensor do exemplo acima produziu a seguinte mensagem no tópico `sensor/32/pressao`:

```python
 {
 	"source": "sensor",
 	"name": "32",
 	"type": "mode-r2d2",
 	"body": {
 		"timestamp": "2019-08-17 17:02:15",
 		"dimension": "pressao",
 		"value": 25.533895448246717,
 		"unity": "bar"
 	}
 }
```

Simuladores podem publicar mensagens aleatórias de tempos em tempos.


# Instanciando Simuladores de Sensores

In [2]:
sensor_1 = IoT_sensor(name="1", dimension="Temperatura",     unity="°C",  min_value=25, max_value=35,   pooling_interval=2)
sensor_2 = IoT_sensor(name="2", dimension="Umidade",         unity="%",   min_value=40, max_value=65,   pooling_interval=3)
sensor_3 = IoT_sensor(name="3", dimension="BateriaRestante", unity="%",   min_value=5,  max_value=100,  pooling_interval=4)
sensor_4 = IoT_sensor(name="4", dimension="SinalWifi",       unity="dBm", min_value=-30, max_value=-90, pooling_interval=2)

# Instanciando Componente de Publicação de Mensagens no MQTT

In [3]:
publisher = IoT_mqtt_publisher("localhost", 1883)

Connected.


### Conectando os Componentes 

In [4]:
sensor_1.connect(publisher)
sensor_2.connect(publisher)
sensor_3.connect(publisher)
sensor_4.connect(publisher)

# Escutando o barramento

Abra um terminal e digite:

```bash

mosquitto_sub -t "sensor/#" -v

```

# Construindo um dashboard local

### Criando o consumidor_1 e seus widgets

In [5]:
widget_1 = widgets.FloatProgress(min=0, max=40.0, bar_style='', orientation='vertical'); widget_1_label = widgets.Label()

consumer_1 = IoT_sensor_consumer("localhost",1883,"sensor/1/+")

### Criando o consumidor_2 e seus widgets

In [6]:
widget_2 = widgets.FloatProgress(min=0, max=90.0, bar_style='warning', orientation='vertical'); widget_2_label = widgets.Label()

consumer_2 = IoT_sensor_consumer("localhost",1883,"sensor/2/+")

### Criando o consumidor_3 e seus widgets

In [7]:
widget_3  = widgets.FloatProgress(min=0, max=100.0, bar_style='info', orientation='vertical'); widget_3_label = widgets.Label()

consumer_3 = IoT_sensor_consumer("localhost",1883,"sensor/3/+")

### Criando o consumidor_4 e seus widgets

In [8]:
widget_4  = widgets.FloatProgress(min=-95.0, max=-30, value=-95, bar_style='success', orientation='vertical'); widget_4_label = widgets.Label()

consumer_4 = IoT_sensor_consumer("localhost",1883,"sensor/4/+")

## Organizando os componentes visualmente

In [9]:
separator = widgets.Label(value=" ---------- ")
col_1 = widgets.VBox([widget_1, widget_1_label])
col_2 = widgets.VBox([widget_2, widget_2_label])
col_3 = widgets.VBox([widget_3, widget_3_label])
col_4 = widgets.VBox([widget_4, widget_4_label])
row_1 = widgets.HBox([separator, col_1, separator, col_2, separator, col_3, separator, col_4])
display(row_1)


### Conectando componentes visuais e seus respectivos consumidores

In [10]:
consumer_1.connect(widget_1, widget_1_label)
consumer_2.connect(widget_2, widget_2_label)
consumer_3.connect(widget_3, widget_3_label)
consumer_4.connect(widget_4, widget_4_label)

# Consultando InfluxDB

In [11]:
! influx --execute "show databases"

name: databases
name
----
telegraf
_internal
iot_example
exemplo_iot


In [12]:
! influx -execute "show measurements" -database 'exemplo_iot'

name: measurements
name
----
cliente_A


In [ ]:
! influx -execute "show measurements" -database 'exemplo_iot'

In [13]:
! influx -execute "show series" -database 'exemplo_iot'

key
---
cliente_A,medida=BateriaRestante,sensor_id=3
cliente_A,medida=SinalWifi,sensor_id=4
cliente_A,medida=Temperatura,sensor_id=1
cliente_A,medida=Umidade,sensor_id=2


In [14]:
! influx -execute "show tag keys" -database 'exemplo_iot'

name: cliente_A
tagKey
------
medida
sensor_id


In [17]:
! influx -execute 'show tag values with key = "sensor_id"' -database 'exemplo_iot'

name: cliente_A
key       value
---       -----
sensor_id 1
sensor_id 2
sensor_id 3
sensor_id 4


In [19]:
! influx -execute 'show field keys ' -database 'exemplo_iot'

name: cliente_A
fieldKey fieldType
-------- ---------
value    float


In [25]:
! influx -execute "select value from cliente_A where medida = 'Temperatura' limit 10 " -database 'exemplo_iot'

name: cliente_A
time                value
----                -----
1597455307220263000 25.48
1597455309222154000 27.75
1597455311224471000 31.08
1597455313227832000 34.37
1597455315228471000 33.52
1597455317230489000 30.85
1597455319232240000 25.36
1597455321232527000 28.88
1597455323233584000 29.08
1597455325236517000 33.38


In [26]:
! influx -execute "select value from cliente_A where medida = 'Umidade' AND time >  now() - 5s " -database 'exemplo_iot'

name: cliente_A
time                value
----                -----
1597455649467183000 48.55


In [27]:
! influx -execute "select max(value), min(value), mean(value) from cliente_A where medida = 'Temperatura' AND time >  now() - 30s " -database 'exemplo_iot'

name: cliente_A
time                max   min   mean
----                ---   ---   ----
1597455674953003439 34.91 25.06 30.149285714285707


In [ ]:

mybinder_urls: ['hub.gke.mybinder.org', ]


In [ ]:
url_message='<b>Grafana interface is available at:</b> <a href="http://{0}{1}{2}" target="_blank"> http://{0}{1}{2}</a>'
display(HTML(url_message.format(os.environ['SERVER_URL'],os.environ['JUPYTERHUB_SERVICE_PREFIX'],os.environ['PROXY_SUBURL'])))
display(HTML('<b>User</b>: admin <br/><b>Password:</b> datasociety'))
display(HTML('<b><font color="red">This cell will keep the Grafana server running. Please do not stop it unless you need to.</font></b>'))

!export GF_SERVER_ROOT_URL="${JUPYTERHUB_SERVICE_PREFIX}${PROXY_SUBURL}" && \
 export GF_SECURITY_ADMIN_PASSWORD=datasociety && \
 cd ${GRAFANA_PATH} && \
 ./bin/grafana-server -convey-silent >> /dev/null